# Talk Like IDEO: similarity scores

In [ ]:
# Jupyter magic
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
# imports
import pandas as pd
import pathlib
import spacy
import collections
from textblob import TextBlob
import gensim
import matplotlib
import numpy as np
import string
import matplotlib.pyplot as plt
from nltk.tokenize import word_tokenize
from gensim.corpora.textcorpus import TextCorpus

In [ ]:
import seaborn as sns

### Load articles: REDO WITH REAL DATA

These were mentioned by Kathy in the [brief](https://docs.google.com/document/d/1ncZfFND1ytnZ3kgMl8RC4GtBFkC3-tAlBDQxuFhRgAU/edit). I manually copied and pasted the text into dropbox to make some quick data.

In [67]:
data_dir = '../1_data/example_articles/'
p = pathlib.Path(data_dir)

articles = {}
for article_path in p.glob('*'):
    print(article_path)
    if article_path.is_dir():
        continue
    
    name = article_path.name
    txt = article_path.read_text()
    articles[name] = txt
    
    print(f'{name}    - has {len(txt)} chars')

../1_data/example_articles/orbia_2019_forbes
orbia_2019_forbes    - has 6585 chars
../1_data/example_articles/orbia_2019_howtolead
orbia_2019_howtolead    - has 6753 chars
../1_data/example_articles/orbia_2018_reuters_businessnews
orbia_2018_reuters_businessnews    - has 2376 chars
../1_data/example_articles/orbia_2019_quartzatwork
orbia_2019_quartzatwork    - has 20006 chars
../1_data/example_articles/.ipynb_checkpoints
../1_data/example_articles/marvin_2017_pressrelease.txt
marvin_2017_pressrelease.txt    - has 4284 chars
../1_data/example_articles/orbia_2018_earnings
orbia_2018_earnings    - has 31100 chars


In [ ]:
ideo_text = articles['orbia_2018_earnings']
other_text = articles['orbia_2019_howtolead']

In [ ]:
ideo_articles = ideo_text.split('\n')
len(ideo_articles)

## Make gensim dictionary

gensim object that lets us do doc2bow functions.

Keeps track of tokens + ids for each token

In [ ]:

s = [word_tokenize(s) for s in ideo_articles]
# ideo_words = word_tokenize(ideo_text)
dictionary = gensim.corpora.Dictionary(s)
dictionary

# potential weakness: not recognizing words we haven't used yet.

In [69]:
# dictionary.token2id


tc = TextCorpus("../1_data/example_articles/marvin_2017_pressrelease.txt")
tc

In [75]:
for t in tc.get_texts():
    print(t)
    break

['paul', 'marvin', 'named', 'ceo', 'marvin', 'companies']


## Make "corpus"

corpus is a list where each element is a BOW (list with word frequencies)

In [ ]:
# corpus = []
# for name, text in articles.items():
#     t2 = word_tokenize(text)
#     corpus.append(dictionary.doc2bow(t2))

def text_to_bow(text):
    tokens = word_tokenize(text)
    return dictionary.doc2bow(tokens)

    
ideo_bow = text_to_bow(ideo_text)


In [ ]:
# def translate_bow_to_english(bow):

# for doc in tf_idf[corpus]:
#     print([[dictionary[id], np.around(freq, decimals=2)] for id, freq in doc])
# #     print(doc)
#     break

## Make similarity model

Term Frequency – Inverse Document Frequency(TF-IDF) is also a bag-of-words model but unlike the regular corpus, TFIDF down weights tokens (words) that appears frequently across documents.

In [ ]:
tf_idf = gensim.models.TfidfModel([ideo_bow])
tf_idf

In [ ]:
help(gensim.similarities.Similarity)

In [ ]:
sims = gensim.similarities.Similarity('workdir/',
                                      tf_idf[ideo_bow],
                                      num_features=len(dictionary))

In [ ]:
other_bow = text_to_bow(other_text)
other_bow[1]

In [ ]:
similarity_array = sims[tf_idf[other_bow]]
len(similarity_array)

In [ ]:
sim_df = pd.DataFrame(all_sims, index=articles.keys(), columns=articles.keys())

In [ ]:
order = good + bad
fig, ax = plt.subplots(figsize=(7, 6))
sns.heatmap(sim_df.loc[order, order], vmax=0.5, annot=True)
plt.tight_layout()
plt.savefig('example_orbia_similarity.png')

In [ ]:
order

In [ ]:
pwd